# Data Bases Project

## Fake and Real News Propagation Analysis

## 1. INTRODUCTION

The goal of this project is to analyse a real and fake news propagation using a database. The idea is to get to know what is inside the dataset by using SQL queries.



We start by loading all useful python libraries for this task.

In [66]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics, model_selection
from sklearn.datasets import fetch_20newsgroups
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import NMF, LatentDirichletAllocation
#from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae


import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.collocations import *
from nltk import tokenize

import networkx as nx
import pandas as pd
from gensim.models import Word2Vec
import seaborn as sns # Seaborn is a Python data visualization library based on matplotlib
import itertools as it
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import os
import random as rdm
import lightgbm as lgb
import xgboost as xgb


#For the data base part
import sys
import sqlite3

%matplotlib inline


## 2. Data Description

The dataset contains a set of news tagged either real or fake (called training set) as well a set 
of untagged news to be predicted correctly by the Web Mining part of the project (called the 
test set). The data also contains how the news (whether fake or real) was propagated through 
twitter with the users who propagated it and how users are connected to one another. 

There are four available CSV-like txt files, 3 of them will be useful (labels_training.txt, 
newsUser.txt, UserUser.txt) as well as two folders with a total of 140 text files (such as 
101.txt). 

These data sets  are described as follows:

--training: this directory contains the content of the news in the training set. Each file is a news stored in the txt format and contains the title, the summary and the content of the news. The name of the txt file is @news_id.txt where @news_id is the identifier of the news.

--test: this directory contains the content of the news in the test set. The news are represented in the same format than for the training directory.

--newsUser.txt: the news-user relationship. For example, '240 1 1' means news 240 is posted/spreaded by user 1 for 1 time.

--UserUser.txt: the user-user relationship. For example, '1589 1' means user 1589 is following user 1;

--labels_training.txt: indicate whether the news in the training set is fake (1) or real (0). For example '23 0' means the news 23 is real.

Below, we load these data sets.
 

In [2]:
pathTrainingSet = 'E:/TSE/S2/web_mining/kaggle_project/fake-news-prediction-toulouse/data_competition/news/training/'
pathTestSet = 'E:/TSE/S2/web_mining/kaggle_project/fake-news-prediction-toulouse/data_competition/news/test/'
userUserPath = 'E:/TSE/S2/web_mining/kaggle_project/fake-news-prediction-toulouse/data_competition/UserUser.txt'
newsUserPath = 'E:/TSE/S2/web_mining/kaggle_project/fake-news-prediction-toulouse/data_competition/newsUser.txt'
labelTrainPath = 'E:/TSE/S2/web_mining/kaggle_project/fake-news-prediction-toulouse/data_competition/labels_training.txt'

A good question is to know how to get what is inside our data set using SQLite. Our main idea is to first build a graph to store all useful informations(news, users, their features and characteristics, relation between users and news, users and users, etc), then make a text mining analysis on this data sets; This will be in the web mining part. Finally in the Data bases part, some of these useful informations to construct our data bases and use SQLite queries to get what is inside.

We start by the web mining part.

## 2. WEB MINING   EXPLORATION

### 2.1 GRAPH MINING

In this section, we create a graph and store all users which share labeled and unlabeled news. Each user node has as attribute :
- followers : indicating the total number of users following current user,
- follows : indicating the total number of users followed by current user,
- totalShare : indicating the total number of news shared by current user,
- totalFake : indicating the total number of fake news among all news shared by current user, 
- totalDiffShare : indicating the total number of different news shared by current user,
- labeled : indicating the total number of labeled news among current user totalDiffShare news
- totalDiffFake : indicating the total number of fake news among user labeled news

In [3]:
g = nx.DiGraph()

usersId = []
with open(userUserPath, 'r') as userUser :
    for users in userUser :
        user1 = 'u' + users[:users.index('\t')]
        user2 = 'u' + users[users.index('\t')+1:len(users)-1]
        
        if user1 not in g.nodes :
            g.add_node(user1, nodeType = 'user', followers = 0, follows = 0, totalShare = 0, totalFake = None,
                       totalDiffShare = 0, labeled = None, totalDiffFake = None)
            usersId.append(user1)
        g.node[user1]['follows'] += 1
            
        
        if user2 not in g.nodes() :
            g.add_node(user2, nodeType = 'user', followers = 0, follows = 0, totalShare = 0, totalFake = None,
                       totalDiffShare = 0, labeled = None, totalDiffFake = None)
            usersId.append(user2)
        g.node[user2]['followers'] += 1
            
        
        if not g.has_edge(user1, user2) :
            g.add_edge(user1, user2, edgeType = 'user edge')

We load in the same graph news with these attributes :
- view : indicating the total number of time the news has been seen,
- share : indicating the total number of time news has been shared,
- soloUser : juste to know the presence of isolate user
- fake : to know text label. None for unkown label and 1 for fake news, 0 for good news,
- topics : to store news topics by importance

Note that we specify node type which is not very necessary here because nodes names already makes difference among them.
We also store users names and news names respectively in two lists to ease manipulating them.

In [4]:
newsUser = pd.read_csv(newsUserPath, sep = '\t', header=None)
labelTrain = pd.read_csv(labelTrainPath, sep = ',', header=0, index_col=0)
newsList = []
soloUser = []
userNews = dict()
labeledNews = ['n'+str(index) for index in list(labelTrain.index)]

for i in range(newsUser.shape[0]) :
    news = 'n' + str(newsUser[0][i])
    user = 'u' + str(newsUser[1][i])
    sharingOccurency = newsUser[2][i]
    if user not in list(userNews.keys()) :
        userNews[user] = list()
    
    if not g.has_edge(news, user) :
        userNews[user].append(news)
        if news not in newsList :
            g.add_node(news, nodeType = 'news', view = 0, share = 0, soloUser = 0, fake = None, topics = list())
            if news in labeledNews :
                g.node[news]['fake'] = labelTrain.at[int(news[1:]), 'class']
            newsList.append(news)
        g.node[news]['share'] += sharingOccurency
        g.node[news]['view'] += (sharingOccurency*g.node[user]['followers'] + 1)
    
        if user not in usersId :
            g.add_node(user, nodeType = 'user', followers = 0, follows = 0, totalShare = 0, totalFake = None,
                       totalDiffShare = 0, labeled = None, totalDiffFake = None)
            g.node[news]['soloUser'] += 1
            soloUser.append(user)
            usersId.append(user)
        g.node[user]['totalShare'] += sharingOccurency
        g.node[user]['totalDiffShare'] += 1
        if g.node[news]['fake'] != None :
            if g.node[user]['totalFake'] == None :
                g.node[user]['totalFake'] = 0
            
            if g.node[user]['totalDiffFake'] == None :
                g.node[user]['totalDiffFake'] = 0
            
            if g.node[news]['fake'] == 1 :
                g.node[user]['totalFake'] += sharingOccurency
                g.node[user]['totalDiffFake'] += 1
            
        g.add_edge(news, user, edgeType = 'news edge')
        if news in labeledNews :
            if g.node[user]['labeled'] == None :
                g.node[user]['labeled'] = 0
            g.node[user]['labeled'] += 1
        
unLabeledNews = list(set(newsList) - set(labeledNews))

Now that we have our graph with its nodes and features, let us now move to the text mining analysis part.

### 2.2  TEXT MINING

The aim of this section is to make text mining analysis on our data sets. We start by loading the whole news in a dictionary called 'data'.

In [26]:
def buildText(textList, wordToDel) :
    '''
    function to read text
    '''
    result = ' '
    for sentence in textList :
        if sentence != wordToDel :
            result = result + sentence[:len(sentence)-2]
    return result



#reading training set
dataTraining = {}
dataNewsTrain = {}
for file in os.listdir(pathTrainingSet):
    iD = 'n' + file[:len(file)-4]
    with open(pathTrainingSet+file, 'r', encoding="utf8") as f :
        dataNewsTrain[iD] = f.readlines()
        dataTraining[iD] = buildText(f.readlines(), '\n')
        


#reading test set
dataTest = {}
dataNewsTest = {}
for file in os.listdir(pathTestSet):
    iD = 'n' + file[:len(file)-4]
    with open(pathTestSet+file, 'r', encoding="utf8") as f :
        dataNewsTest[iD] = f.readlines()
        dataTest[iD] = buildText(f.readlines(), '\n')
        
dataNews = dataNewsTrain.copy()
dataNews.update(dataNewsTest)

data = dataTraining.copy()
data.update(dataTest)

#### 2.2.1  Cleaning the data set

In this section, we clean the data. For this purpose:
- we first remove the stopwords;
- then, we stem all remaining words, that is to keep only each remaining word root.

In [29]:
stopWords = stopwords.words('english')
stemming = PorterStemmer()
tokenizer = tokenize.RegexpTokenizer(r'\w+')

def dataFilter(data, stopWords) :
    '''
    Function to filter data and only keep important words
    '''
    returnValue = {}
    for key in data.keys() :
        returnValue[key] = [stemming.stem(word) for word in tokenizer.tokenize(data[key].lower()) if word not in stopWords and word.isalpha()]
    return returnValue


tokenazation = dataFilter(data, stopWords)

#setting up the cleaned data
cleanData = {}
for key in tokenazation.keys() :
    cleanData[key] = ' '.join(tokenazation[key])

cleanDataTraining = {key : cleanData[key] for key in dataTraining.keys()}
cleanDataTest = {key : cleanData[key] for key in dataTest.keys()}


posTag = [nltk.pos_tag(tokenazation[key]) for key in tokenazation.keys()]

sPosTag = pd.Series()
for p in posTag :
    sPosTag = sPosTag.append(pd.Series(p))
    
#sPosTag.value_counts()

Removing duplicate elements from the list of users and news if they exist.

In [30]:
# Python code to remove duplicate elements 
def Remove(duplicate): 
    finalList = [] 
    for elt in duplicate: 
        if elt not in finalList: 
            finalList.append(elt) 
    return finalList

newsList = Remove(newsList)
usersId = Remove(usersId)

len(newsList), len(usersId), len(dataNews.keys())

(240, 23865, 240)

## 3. DATA BASES EXPLORATION

### 3.1  Building the Data Sets

In this section, we build the 5 data sets that will be used to construct our data bases. They are build as list of tuples. This is done in order to facilitate their insertion into tables. We create our ids as text in order to differentiate user ids to news ids. For instance:
- 'u1' represent the id of the user 1,
- 'n1' represent the id of the news 1.
For each data set created below, each tuple represent a row of the corresponding table in our data bases.

#### 3.1.1  The News Data Set

We create here a list of tuples of the form:(newsId, newsTitle, newsText), where newsTitle and newsText are the corresponding text and title of the newsId.

In [448]:
newsData = []
for newsId in dataNews.keys():
    newsTitle = dataNews[newsId][0]
    newsText = buildText(dataNews[newsId][1:], '\n')
    newsElt = (newsId, newsTitle, newsText)
    newsData.append(newsElt)
    

#The first 2 elements of the list
newsData[0:2]

[('n1',
  'Trump Just Insulted Millions Who Lost Everything In Bush\'s Recession, "That\'s Just Business"\n',
  ' 16.8k SHARES SHARE THIS STORHillary Clinton just called out the fact that Donald Trump cheered for the housing crisis in anticipation of its collapse – which is absolutely true. Trump told The Globe and Mail in March of 2007: “People have been talking about the end of the cycle for 12 years, and I’m excited if it is. I’ve always made more money in bad markets than in good markets.In fact, Trump thought the housing crisis was much-ado-do about nothing for high-end investments, and told investors: “I don’t see the subprime problems affecting the higher-end stuff…In fact, he is advising investors that there are now great deals in buying subprime mortgages at a discount and repossessed houses at low prices.Of course, on one level, Trump wasn’t wrong\xad\xad—in that the subprime crisis affected mostly poorer Americans who lost their homes and jobs and the ability to support thei

#### 3.1.2  The Labels Data Set

The labels of news can take 3 values:
- 0 if the news is real,
- 1 if the news is fake,
- 2 if the label news is unknown.

We create here a list of tuples of the form:(newsId, newsLabel), where newsLabel is the corresponding label of the newsId.

In [218]:
newsLabelsData = []
for newsId in dataNews.keys():
    if str(g.node[newsId]['fake']) in ['0', '1']:
        newsLabelsElt = (newsId, str(g.node[newsId]['fake']))
        newsLabelsData.append(newsLabelsElt)
    else:
        newsLabelsElt = (newsId, str(2))
        newsLabelsData.append(newsLabelsElt)

#The first 3 elements of the list
newsLabelsData[0:3]

[('n1', '0'), ('n10', '0'), ('n100', '0')]

#### 3.1.3  The Users Data Set

We create here a list of tuples of the form:(userId,), where:
 - userId is the Id of each user,
 - usertotalNbFake : indicate the total number of fake news shared by user with id 'userId'. We set it to be -1 for userId whose the label of news shared is unknown, so for user that we don't know if their news shared is fake or not; For others users, it is a number greater than or equal to 0.

In [308]:
usersData = []
for userId in usersId:
    if g.node[userId]['totalFake'] == None:     
        usertotalNbFake = -1                 #user who did share any think
        userElt = (userId, usertotalNbFake)
        usersData.append(userElt)
    else:
        usertotalNbFake = int(g.node[userId]['totalFake'])
        userElt = (userId, usertotalNbFake)
        usersData.append(userElt)

#The first 3 elements of the list
usersData[0:4]

[('u507', 1), ('u1', 1), ('u1589', 1), ('u5307', 1)]

#### 3.1.4  The PropagationNews Data Set

We create here a list of tuples of the form:(newsId, UsersIdsFollowers, nbNewsShare, nbNewsView), where:
- usersIdsFollowers is the list of users that follows the news with id 'newsId';
- nbNewsShare is the number of times this news was shared;
- nbNewsView is the number of view of this news; This is compute in the graph mining part by using the number of time the news was shared and the number of followers of each user who shared that news.

In [309]:
newsPropagationData = []
for newsId in list(dataNews.keys()):
        nbNewsShare = int(g.node[newsId]['share'])
        nbNewsView = int(g.node[newsId]['view'])
        newsPropagationElt = (newsId, str(list(g.neighbors(newsId))),nbNewsShare,nbNewsView)
        newsPropagationData.append(newsPropagationElt)

#The first 2 elements of the list
newsPropagationData[0:2]

[('n1',
  "['u105', 'u1027', 'u1592', 'u2486', 'u2787', 'u2848', 'u4301', 'u4306', 'u4843', 'u5982', 'u6918', 'u7113', 'u8846', 'u9351', 'u9872', 'u10065', 'u10665', 'u11676', 'u11821', 'u13163', 'u14545', 'u14828', 'u17022', 'u17117', 'u17656', 'u18992', 'u19013', 'u19826', 'u20935', 'u21128', 'u21507', 'u22946', 'u23091', 'u23293', 'u23453']",
  36,
  2139),
 ('n10',
  "['u191', 'u1772', 'u1925', 'u1942', 'u3638', 'u4258', 'u4429', 'u7158', 'u7200', 'u7500', 'u8339', 'u9135', 'u10009', 'u10798', 'u11933', 'u13816', 'u15498', 'u15713', 'u16302', 'u16964', 'u18747', 'u20499', 'u22207', 'u23105', 'u23272', 'u23445']",
  26,
  529)]

In [123]:
int(2)

2

#### 3.1.5  The Followers Data Set

We create here a list of tuples of the form:(userId, usersIdsFollowed, nbFollowers), where:
- userId is the id for each user; 
- usersIdsFollowers is list of ids of the followers of the user with id 'userId';
- nbFollowers is the number of followers of the user with id 'userId'; 
To build this list of tuples, we first use our csv dataset called 'userUserPath', to create a list of ids of followers of each user. This list is called 'followersIdList'. More precisely, 'followersIdList[i]' contains the list of ids of followers of the user with id 'ui'.

In [64]:
userUserList = []
userUser2 = []    #list of users followed
m = [] # list of number of user following a user in the list of users followed
k = 0 # number of user followed
with open(userUserPath, 'r') as userUser :
    for users in userUser :
        user1 = 'u' + users[:users.index('\t')]
        user2 = 'u' + users[users.index('\t')+1:len(users)-1]
        userUserList.append([user1, user2])
        if user2 not in userUser2:
            userUser2.append(user2) 
            m.append(1)
            k += 1 
        else:
            m[k-1] += 1

            
# this works only since the list of couple of user is ordered by the user they follow    
# e.g. #507 	1, #13267	1 ,#1205	2, #15442	2, #17464	2, ...
followersIdList = [[] for i in range(len(userUser2))]
j=0 # j is the index running trough the list of couple of user     
p = 0
while j < len(userUserList):
    for k in range(len(m)): # k is the index runing through the list of users followed
        if userUserList[j][1] == userUser2[k]:
            for l in range(m[k]):
                followersIdList[k].append(userUserList[j+l][0])
            break
    j = j+ m[k]
    p +=1;


Once this list is constructed, we use it to build our data set 'followersData'. 

In [65]:
# we use the list followersIdList.
followersData1 = []         #for users that are followed
for  l in range(len(userUser2)):
        followersElt = (userUser2[l], str(followersIdList[l]), m[l])
        followersData1.append(followersElt)


usersIdNotFollowed = list(set(usersId)-set(userUser2))

followersData2 = []         #for users that are not followed
for  l in range(len(usersIdNotFollowed)):
        followersElt = (usersIdNotFollowed[l], str([]), 0)
        followersData2.append(followersElt)

#Concatenate the two list
followersData = followersData1 + followersData2
#The first 3 elements of the list
print(followersData[0:2])

[('u1', "['u507', 'u1589', 'u5307', 'u11421', 'u13267', 'u20571']", 6), ('u2', "['u1205', 'u2631', 'u8114', 'u10429', 'u15442', 'u17464', 'u19399', 'u19991', 'u20371', 'u22113', 'u23687']", 11)]


### 3.2  Building our data base


In this section, we would like to construct our data base. For this purpose, we first create an empty data base called 'projectdatabd.db', then we create 5 empty tables and insert all tuples of each of the 5 data sets created above in its corresponding table.

Below, we have the creation of our empty data bases and a we define a function that takes our query (stored as a string) as an input and shows the result as a formatted dataframe. This was done so that we can see the results of your queries in a neatly formatted table.

In [449]:
con = sqlite3.connect('projectdataBD.db')  #open up a connection to our SQLite DB  
c = con.cursor()                #create a cursor  to execute SQL statements

def runQuery(query):
    return pd.read_sql_query(query,con)

 #### 3.2.1 Creating the tables
 
 Here, we first define the tables of the data base with their corresponding variables and types. This is done by using a dictionary called 'queriesSql' that have as key either 'createTableName' or 'dropTableName' and as value a string with the corresponding lines of table's creations(or dropping) in SQLITE.

In [451]:
#dictionary that contains all queries(tables)
queriesSql = {}

#news table
queriesSql['createNews']   = '''
                                create table if not exists news(
                                    newsId   text,
                                    newsTitle    text,
                                    newsText    text,
                                    --primary key
                                    CONSTRAINT pk_news PRIMARY KEY (newsId)
                                )
                                '''
queriesSql['dropNews']     = '''drop table if exists news'''

#news_labels table
queriesSql['createNewsLabels']   = '''
                                create table if not exists newsLabels(
                                    newsId   text,
                                    label    text,
                                    --primary key
                                    CONSTRAINT pk_newsLabels PRIMARY KEY  (newsId)
                                    --foreign key
                                    CONSTRAINT fk_newsLabels_news FOREIGN KEY (newsId) 
                                        REFERENCES news (newsId)
                                )
                                '''
queriesSql['dropNewsLabels']     = '''drop table if exists newsLabels'''

#users table
queriesSql['createUsers']   = '''
                                create table if not exists users(
                                    userId   text,
                                    usertotalNbFake  int,
                                     --primary key
                                    CONSTRAINT pk_users PRIMARY KEY  (userId)
                                )
                                '''
queriesSql['dropUsers']     = '''drop table if exists users'''

#newsPropagation table
queriesSql['createNewsPropagation']   = '''
                                create table if not exists newsPropagation(
                                    newsId       text,
                                    usersIdsFollowers   text, --list of user ids of users that follows the newsId
                                    nbNewsShare      int, --that's PropagCount(the number of time the news was shared) 
                                    nbNewsView       int,     --number of time the news was view
                                     --primary key
                                    CONSTRAINT pk_newsPropagation PRIMARY KEY  (newsId)
                                    --foreign key
                                    CONSTRAINT fk_newsPropagation_news FOREIGN KEY (newsId) 
                                        REFERENCES news (newsId)
                                )
                                '''
queriesSql['dropNewsPropagation']     = '''drop table if exists newsPropagation'''

#followers table
queriesSql['createFollowers']   = '''
                                create table if not exists followers(
                                    userId      text,
                                    userIdsFollowed   text,  --list of ids of the followers of userId
                                    nbFollowers     int,
                                         --primary key
                                    CONSTRAINT pk_followers PRIMARY KEY  (userId)
                                        --foreign key
                                    CONSTRAINT fk_followers_users FOREIGN KEY (userId)
                                        REFERENCES users (userId)
                                )
                                '''
queriesSql['dropFollowers']     = '''drop table if exists followers'''

In [452]:
createQueries  = ['createNews', 'createNewsLabels', 
                   'createUsers', 'createNewsPropagation', 'createFollowers']

dropQueries    = ['dropFollowers', 'dropNewsPropagation', 'dropUsers',
                   'dropNewsLabels','dropNews']

list(queriesSql.keys())

['createNews',
 'dropNews',
 'createNewsLabels',
 'dropNewsLabels',
 'createUsers',
 'dropUsers',
 'createNewsPropagation',
 'dropNewsPropagation',
 'createFollowers',
 'dropFollowers']

Now, we create our tables in our data base. We use the following function to put all tables in the database

In [453]:
def fillqueries(queriesSqlDict, queriesList, c):
    #fill all queries from the database
    for queryName in queriesList:
    #get my query inside my dictionary of queries        
        c.execute(queriesSqlDict[queryName])

#drop all tables from the database
fillqueries(queriesSql, dropQueries, c)
    
#create all tables in the database
fillqueries(queriesSql, createQueries, c)
    

 #### 3.2.2 Filling the tables

Here, we fill out each table created above using its corresponding data set. To do so, we define the following function that take as input a list of tuples, the tablename and the cursor c, then insert each tuple of this list in data base table  named 'tablename'.


In [454]:
def dataListToDb (dataList, tablename, c):
    '''
    inputs: dataList(list of tuples), the tablename, the cursor c
    
    Insert each tuple of this list in data base table with named 'tablename'
    
    '''
    for dataListElt in dataList:
        # Insert the dataListElt!
        #print("Inserting the file row number", dataList.index(dataListElt))
        value = '  VALUES(%s)' % ','.join('?' * len(dataListElt))
        toInsert = str('INSERT INTO  ') + str(tablename) + str(value)
        c.execute(toInsert, dataListElt)
      

We use the above function to insert the data sets into the tables

In [455]:
#news
dataListToDb(newsData, 'news', c)

In [456]:
#newsLabels
dataListToDb(newsLabelsData, 'newsLabels', c)

In [457]:
#users
dataListToDb(usersData, 'users', c)

In [458]:
#newsPropagation
dataListToDb(newsPropagationData, 'newsPropagation', c)

In [459]:
#followers
dataListToDb(followersData, 'followers', c)

In [460]:
con.commit()
    
#closing the connection
#conn.close()

This is our data bases dictionary.


<img src="dbdictionary.PNG">

### 3.3  Exploration of the data base

### 3.3.1 Initial row information

Here, we list for each table the total number of rows. This is to get for instance, the total number of news, the total number of users;

From the following, we get that we have a total number of 240 news and 23 865 users in our data bases. The total number of rows of others tables are given below. The first column correspond the the index of the line(so 0 is the index of our computation). Python create by default table with first columns as index.

In [461]:
countNews = 'select count(*) as  nbNews from news'
runQuery(countNews)

,nbNews
0,240


In [462]:
countLabels = 'select count(*) as nbLabels from newsLabels'
runQuery(countLabels)

,nbLabels
0,240


In [463]:
countUsers = 'select count(*) as  nbUsers from users'
runQuery(countUsers)

,nbUsers
0,23865


In [464]:
countPropagation = 'select count(*) as nbPropagationLines from newsPropagation'
runQuery(countPropagation)

,nbPropagationLines
0,240


In [465]:
countfollowers = 'select count(*) as nbFollowersLines from followers'
runQuery(countfollowers)

,nbFollowersLines
0,23865


### 3.3.1 Description of the contains of the data base 

Here, we describe the content of the data base by giving some precise informations on the contains of each table.

We start by printing the first 3 rows of each table. This is done below. The first column of each of these printed table correspond  the index of the line(so 0 is the index of our computation). Python create by default table with first columns as index.

In [466]:
headNews = 'select * from news limit 3'
runQuery(headNews)

,newsId,newsTitle,newsText
0,n1,Trump Just Insulted Millions Who Lost Everythi...,16.8k SHARES SHARE THIS STORHillary Clinton j...
1,n10,Famous dog killed in spot she waited a year fo...,Famous dog killed in spot she waited a year f...
2,n100,House oversight panel votes Clinton IT chief i...,Members of the House Oversight and Government...


In [467]:
headNewsLabels = 'select * from newsLabels limit 3'
runQuery(headNewsLabels)

,newsId,label
0,n1,0
1,n10,0
2,n100,0


In [468]:
headUsers = 'select * from users limit 3'
runQuery(headUsers)

,userId,usertotalNbFake
0,u507,1
1,u1,1
2,u1589,1


In [469]:
headNewsPropagation = 'select * from newsPropagation limit 3'
runQuery(headNewsPropagation)

,newsId,usersIdsFollowers,nbNewsShare,nbNewsView
0,n1,"['u105', 'u1027', 'u1592', 'u2486', 'u2787', '...",36,2139
1,n10,"['u191', 'u1772', 'u1925', 'u1942', 'u3638', '...",26,529
2,n100,"['u175', 'u824', 'u1141', 'u1635', 'u2348', 'u...",64,1408


In [470]:
headfollowers = 'select * from followers limit 3'
runQuery(headfollowers)

,userId,userIdsFollowed,nbFollowers
0,u1,"['u507', 'u1589', 'u5307', 'u11421', 'u13267',...",6
1,u2,"['u1205', 'u2631', 'u8114', 'u10429', 'u15442'...",11
2,u3,"['u698', 'u2541', 'u4043', 'u4782', 'u5250', '...",48


Concerning the news, we first look at the number of news per label.

In [471]:
newsBylabel = '''SELECT  label as 'News label', count(label) as 'CountBylabel'
                    FROM newsLabels
                    group by label;
                    '''

runQuery(newsBylabel)

,News label,CountBylabel
0,0,96
1,1,97
2,2,47


Then we look the news that have a higher number of share(propagCount).

In [473]:
newshighestNbShared = '''
                        select newsId,  nbNewsShare
                            from newsPropagation
                            order by nbNewsShare desc
                            limit 10;
                        '''
runQuery(newshighestNbShared)

,newsId,nbNewsShare
0,n178,3437
1,n227,3383
2,n233,1573
3,n155,1438
4,n37,1044
5,n162,1030
6,n128,1029
7,n135,824
8,n145,682
9,n204,638


The news with the  lower number of share(propagCount) are the following:

In [474]:
newshighestNbShared = '''
                        select newsId, nbNewsShare
                            from newsPropagation
                            order by nbNewsShare desc
                            limit 10;
                        '''
runQuery(newshighestNbShared)

,newsId,nbNewsShare
0,n178,3437
1,n227,3383
2,n233,1573
3,n155,1438
4,n37,1044
5,n162,1030
6,n128,1029
7,n135,824
8,n145,682
9,n204,638


The news that have a higher number of view are the following:

In [475]:
newshighestNbView = '''
                        select newsId, nbNewsView
                            from newsPropagation
                            order by nbNewsView desc
                            limit 10;
                        '''
runQuery(newshighestNbView)

,newsId,nbNewsView
0,n227,85732
1,n178,80730
2,n233,41917
3,n155,34959
4,n37,27028
5,n128,24872
6,n162,21775
7,n135,19944
8,n145,19345
9,n204,17420


The news with the lower number of view are the following:

In [476]:
newslowestNbView = '''
                        select newsId, nbNewsView
                            from newsPropagation
                            order by nbNewsView asc
                            limit 10;
                        '''
runQuery(newslowestNbView)  

,newsId,nbNewsView
0,n127,3
1,n173,3
2,n198,7
3,n225,8
4,n22,9
5,n117,18
6,n103,21
7,n104,24
8,n77,28
9,n120,29


Few news that are fake.

In [477]:
newsLabelFake = '''
                        select newsId, label as labelFake
                            from newsLabels
                            where label = '1'
                            limit 10;
                        '''
runQuery(newsLabelFake)

,newsId,labelFake
0,n121,1
1,n122,1
2,n123,1
3,n124,1
4,n126,1
5,n127,1
6,n130,1
7,n131,1
8,n132,1
9,n134,1


In [ ]:
Few real news.

In [478]:
newsLabelReal = '''
                        select newsId, label as labelReal
                            from newsLabels
                            where label = '0'
                            limit 10;
                        '''
runQuery(newsLabelReal)

,newsId,labelReal
0,n1,0
1,n10,0
2,n100,0
3,n101,0
4,n102,0
5,n103,0
6,n104,0
7,n105,0
8,n106,0
9,n107,0


In [ ]:
Few unlabeled news.

In [479]:
newsUnLabel = '''
                        select newsId, label as labelNone
                            from newsLabels
                            where label = '2'
                            limit 10;
                        '''
runQuery(newsUnLabel)

,newsId,labelNone
0,n113,2
1,n114,2
2,n116,2
3,n117,2
4,n125,2
5,n128,2
6,n129,2
7,n133,2
8,n137,2
9,n144,2


Concerning the users:

We first look at the users id who have a highest number of followers: It is the user with id 'u22926', it has 1522 followers.

In [480]:
userhighestNbFollowers = '''
                        select userId,  userIdsFollowed, nbFollowers
                            from followers
                            where nbfollowers = (SELECT max(nbFollowers) from followers);
                        '''
runQuery(userhighestNbFollowers)

,userId,userIdsFollowed,nbFollowers
0,u22926,"['u1', 'u3', 'u28', 'u34', 'u60', 'u76', 'u81'...",1522


This gives the number of users that have no followers: 1089.

In [481]:
nbUserZeroFollowers = '''
                        select count(*) as nbUserZeroFollowers
                            from followers
                            where nbfollowers = (SELECT min(nbFollowers) from followers)
                            ;
                        '''
runQuery(nbUserZeroFollowers)

,nbUserZeroFollowers
0,1089


This gives the number of users that have some followers: 22776.

In [482]:
nbUserSomeFollowers = '''
                        select count(*) as nbUserSomeFollowers
                            from followers
                            where nbfollowers > (SELECT min(nbFollowers) from followers)
                            ;
                        '''
runQuery(nbUserSomeFollowers)

,nbUserSomeFollowers
0,22776


These are five users with lower number of followers:

In [483]:
usermeanNbFollowers = '''
                        select userId,  userIdsFollowed, nbFollowers
                            from followers
                            where nbfollowers = (SELECT min(nbFollowers) from followers)
                            limit 5;
                        '''
runQuery(usermeanNbFollowers)

,userId,userIdsFollowed,nbFollowers
0,u10218,[],0
1,u2147,[],0
2,u23601,[],0
3,u12191,[],0
4,u17357,[],0


The five users with higher number of followers:

In [484]:
usershighestNbFollowers = '''
                        select userId,  userIdsFollowed, nbFollowers
                            from followers
                            order by nbFollowers desc
                            limit 5;
                        '''
runQuery(usershighestNbFollowers)

,userId,userIdsFollowed,nbFollowers
0,u22926,"['u1', 'u3', 'u28', 'u34', 'u60', 'u76', 'u81'...",1522
1,u11717,"['u28', 'u61', 'u125', 'u126', 'u147', 'u182',...",1470
2,u21298,"['u4', 'u28', 'u34', 'u35', 'u38', 'u99', 'u11...",1456
3,u16022,"['u34', 'u35', 'u56', 'u60', 'u118', 'u126', '...",1336
4,u16321,"['u28', 'u34', 'u35', 'u60', 'u66', 'u81', 'u9...",1249


Users with lowest number of followers including 0

In [485]:
userslowestNbFollowersZeros = '''
                        select userId,  userIdsFollowed, nbFollowers
                            from followers
                            order by nbFollowers asc
                            limit 5;
                        '''
runQuery(userslowestNbFollowersZeros)

,userId,userIdsFollowed,nbFollowers
0,u10218,[],0
1,u2147,[],0
2,u23601,[],0
3,u12191,[],0
4,u17357,[],0


Users with lowest number of followers > 0

In [486]:
userslowestNbFollowers = '''
                        select userId,  userIdsFollowed, nbFollowers
                            from followers
                            where (nbFollowers > 0)
                            order by nbFollowers asc
                            limit 5;
                        '''
runQuery(userslowestNbFollowers)

,userId,userIdsFollowed,nbFollowers
0,u6,['u19399'],1
1,u11,['u6048'],1
2,u19,['u4303'],1
3,u26,['u22113'],1
4,u27,['u12952'],1


Number of users who shared at least one news

In [487]:
nbUserSharedNews = '''
                        select count(*) as nbUsersSharedNews
                            from followers
                            where (nbFollowers >= 1);
                        '''
runQuery(nbUserSharedNews)

,nbUsersSharedNews
0,22776


Number of users who never shared  news

In [488]:
nbUserSharedNews = '''
                        select count(*) as nbUsersSharedNoNews
                            from followers
                            where (nbFollowers == 0);
                        '''
runQuery(nbUserSharedNews)

,nbUsersSharedNoNews
0,1089


### 3.3.2 More Advanced Information on the data bases

Here, we would like to know the users who mostly spread real information.
To find this, we select the columns 'userId' and 'usertotalNbFake' in the table users, then the column 'nbFollowers' from the table followers; We join these three columns and output only the rows with nbFollowers >= 500 and usertotalNbFake = 0.

From the table below, there are 11 users that spread mostly real information compared to others. 
The user with id 'u164' always spread real news to it 1218 followers.

In [489]:
queryUsers1 = '''SELECT users.userId as 'User Id', users.usertotalNbFake  as 'Number of Fake News Shared',
                    followers.nbFollowers as 'Number of followers'
                    FROM users
                    INNER JOIN followers
                    ON users.userId = followers.userId
                    where followers.nbFollowers >= 500 and users.usertotalNbFake = 0
                    --group by users.usertotalNbFake 
                    order by  followers.nbFollowers desc;
                    '''

runQuery(queryUsers1)

,User Id,Number of Fake News Shared,Number of followers
0,u164,0,1218
1,u16907,0,1003
2,u4118,0,909
3,u18175,0,781
4,u20750,0,755
5,u6653,0,678
6,u22630,0,646
7,u4814,0,625
8,u22551,0,621
9,u7514,0,618


Here, we would like to know the users who mostly spread fake information.
To find this, we select the columns 'userId' and 'usertotalNbFake' in the table users, then the column 'nbFollowers' from the table followers; We join these three columns and output only the rows with nbFollowers >= 100 and usertotalNbFake  > 7.

From the table below, there are 11 users that spread mostly real information compared to others. 
The user with id 'u18 542' spread  30 fake news to it 141 followers, so we can say that because of him there are 4230 view of fake news.

In [490]:
queryUsers2 = '''SELECT users.userId as 'User Id', users.usertotalNbFake  as 'Number of Fake News Shared',
                    followers.nbFollowers as 'Number of followers', 
                    users.usertotalNbFake*followers.nbFollowers  as 'Number Fake News view'
                    FROM users
                    INNER JOIN followers
                    ON users.userId = followers.userId
                    where followers.nbFollowers >= 100 and users.usertotalNbFake > 7
                    order by users.usertotalNbFake*followers.nbFollowers  desc;
                    '''

runQuery(queryUsers2)

,User Id,Number of Fake News Shared,Number of followers,Number Fake News view
0,u18542,30,141,4230
1,u20378,27,103,2781
2,u8700,25,108,2700
3,u22540,13,200,2600
4,u3412,10,169,1690
5,u17203,11,132,1452
6,u8271,12,116,1392
7,u15070,12,108,1296
8,u8723,12,100,1200
9,u12895,11,100,1100


Concerning the news, We would like to know the most shared and view fake news. 
For this purpose, we proceed similarly as above but using the tables newsLabels and newsPropagation; And we look for row with label = '1' and nbNewsShare >= 500.
From the table below, there are 9 news satisfying these conditions; The first one is the new 'n227'; it was 3 383 times and has 85732 views. 

In [491]:
queryNews1 = '''SELECT newsLabels.newsId as 'News Id', newsLabels.label as 'News label',
                    newsPropagation.nbNewsShare  as 'Number of Time Shared',
                    newsPropagation.nbNewsView as 'Number of Time View'
                    FROM newsLabels
                    JOIN newsPropagation
                    ON newsLabels.newsId = newsPropagation.newsId
                    where newsLabels.label = '1' and newsPropagation.nbNewsShare >= 500
                    order by  newsPropagation.nbNewsShare desc;
                    '''

runQuery(queryNews1)

,News Id,News label,Number of Time Shared,Number of Time View
0,n227,1,3383,85732
1,n233,1,1573,41917
2,n162,1,1030,21775
3,n135,1,824,19944
4,n204,1,638,17420
5,n221,1,569,13738
6,n124,1,557,15838
7,n219,1,550,14865
8,n142,1,542,13231


We would like to know the most shared real news.
For this purpose, we proceed similarly as above but using the tables newsLabels and newsPropagation; And we look for row with label = '0' and newsPropagation.nbNewsShare >= 300.
From the table below, there are 4 news satisfying these conditions; The first one is the new 'n37'; it was 1 044 times and has 27028 views. 

In [492]:
queryNews2 = '''SELECT newsLabels.newsId as 'News Id', newsLabels.label as 'News label',
                    newsPropagation.nbNewsShare  as 'Number of Time Shared',
                    newsPropagation.nbNewsView as 'Number of Time View'
                    FROM newsLabels
                    JOIN newsPropagation
                    ON newsLabels.newsId = newsPropagation.newsId
                    where newsLabels.label = '0' and newsPropagation.nbNewsShare >= 300
                    order by  newsPropagation.nbNewsShare desc;
                    '''

runQuery(queryNews2)

,News Id,News label,Number of Time Shared,Number of Time View
0,n37,0,1044,27028
1,n59,0,399,9550
2,n94,0,347,9619
3,n70,0,340,9748


Here, we present real news that are not really shared or view.
We proceed again as above but under the condition that:
- label = '0' and nbNewsShare < 6

From this table, the news 'n104' is a real news that was shared only once. It has 24 view may be because of the number of followers of the users who shared it.

In [493]:
queryNews3 = '''SELECT newsLabels.newsId as 'News Id', newsLabels.label as 'News label',
                    newsPropagation.nbNewsShare  as 'Number of Time Shared',
                    newsPropagation.nbNewsView as 'Number of Time View'
                    FROM newsLabels
                    JOIN newsPropagation
                    ON newsLabels.newsId = newsPropagation.newsId
                    where newsLabels.label = '0' and newsPropagation.nbNewsShare < 6
                    order by  newsPropagation.nbNewsShare asc;
                    '''

runQuery(queryNews3)

,News Id,News label,Number of Time Shared,Number of Time View
0,n104,0,1,24
1,n22,0,1,9
2,n101,0,5,34
3,n103,0,5,21
4,n109,0,5,38
5,n120,0,5,29
6,n27,0,5,44
7,n35,0,5,369
8,n77,0,5,28
9,n81,0,5,53


Here, we present real news that are not really view.
We proceed again as above but under the condition that:
- label = '0' and nbNewsView < 30

From this table, There are only 5 such news; the news 'n22' is a real news that was view only 9 time.

In [494]:
queryNews4 = '''SELECT newsLabels.newsId as 'News Id', newsLabels.label as 'News label',
                    newsPropagation.nbNewsView as 'Number of Time View'
                    FROM newsLabels
                    JOIN newsPropagation
                    ON newsLabels.newsId = newsPropagation.newsId
                    where newsLabels.label = '0' and newsPropagation.nbNewsView < 30
                    order by  newsPropagation.nbNewsView asc;
                    '''

runQuery(queryNews4)

,News Id,News label,Number of Time View
0,n22,0,9
1,n103,0,21
2,n104,0,24
3,n77,0,28
4,n120,0,29


Here, we present unlabeled news that are not really shared or view.
We proceed again as above but under the condition that:
- label = '2' and nbNewsShare < 6

From this table, there are 6 such news; the news 'n183' was the least shared, and the news 'n117'  was the least view.

In [495]:
queryNews5 = '''SELECT newsLabels.newsId as 'News Id', newsLabels.label as 'News label',
                    newsPropagation.nbNewsShare  as 'Number of Time Shared',
                    newsPropagation.nbNewsView as 'Number of Time View'
                    FROM newsLabels
                    JOIN newsPropagation
                    ON newsLabels.newsId = newsPropagation.newsId
                    where newsLabels.label = '2' and newsPropagation.nbNewsShare < 6 
                    order by  newsPropagation.nbNewsShare asc;
                    '''

runQuery(queryNews5)

,News Id,News label,Number of Time Shared,Number of Time View
0,n183,2,1,131
1,n202,2,2,133
2,n117,2,3,18
3,n137,2,4,29
4,n218,2,5,61
5,n40,2,5,238


Here, we present unlabeled news that were higly shared or view.
We proceed again as above but under the condition that:
- label = '2' and nbNewsShare > 400

From this table, there are 5 such news; the news 'n178' is an unlabeled news that was shared 3437 times. It has 80 730 view.

In [496]:
queryNews6 = '''SELECT newsLabels.newsId as 'News Id', newsLabels.label as 'News label',
                    newsPropagation.nbNewsShare  as 'Number of Time Shared',
                    newsPropagation.nbNewsView as 'Number of Time View'
                    FROM newsLabels
                    JOIN newsPropagation
                    ON newsLabels.newsId = newsPropagation.newsId
                    where newsLabels.label = '2' and newsPropagation.nbNewsShare > 400
                    order by  newsPropagation.nbNewsShare desc;
                    '''

runQuery(queryNews6)

,News Id,News label,Number of Time Shared,Number of Time View
0,n178,2,3437,80730
1,n155,2,1438,34959
2,n128,2,1029,24872
3,n145,2,682,19345
4,n144,2,413,9729


Now, we would to create a subset of interesting users. For us, interesting users means:
- users who share real news(usertotalNbFake = 0) and have a high number of followers(nbFollowers >= 300)

Below we found them, we proceed similarly as above but by extracting columns that follows the conditions:
- nbFollowers >= 300 and usertotalNbFake = 0; Then we order by number of followers to see the best user.
From the table below, there are 21 good users and the first one is 'u164' with 1218 followers.

In [497]:
goodUsers = '''SELECT users.userId as 'User Id', users.usertotalNbFake  as 'Number of Fake News Shared',
                    followers.nbFollowers as 'Number of followers'
                    FROM users
                    INNER JOIN followers
                    ON users.userId = followers.userId
                    where followers.nbFollowers >= 300 and users.usertotalNbFake = 0
                    order by  followers.nbFollowers desc;
                    '''

runQuery(goodUsers)

,User Id,Number of Fake News Shared,Number of followers
0,u164,0,1218
1,u16907,0,1003
2,u4118,0,909
3,u18175,0,781
4,u20750,0,755
5,u6653,0,678
6,u22630,0,646
7,u4814,0,625
8,u22551,0,621
9,u7514,0,618


Concerning the news, we present real news that are highly shared or view.
We proceed again as above but under the condition that:
- label = '0' and nbNewsShare >= 200

In [498]:
goodNews = '''SELECT newsLabels.newsId as 'News Id', newsLabels.label as 'News label',
                    newsPropagation.nbNewsShare  as 'Number of Time Shared',
                    newsPropagation.nbNewsView as 'Number of Time View'
                    FROM newsLabels
                    JOIN newsPropagation
                    ON newsLabels.newsId = newsPropagation.newsId
                    where newsLabels.label = '0' and newsPropagation.nbNewsShare >= 200
                    order by  newsPropagation.nbNewsShare desc;
                    '''

runQuery(goodNews)

,News Id,News label,Number of Time Shared,Number of Time View
0,n37,0,1044,27028
1,n59,0,399,9550
2,n94,0,347,9619
3,n70,0,340,9748
4,n8,0,257,5605
5,n9,0,257,5605
6,n65,0,249,7964
7,n78,0,207,4850
8,n89,0,201,5163


Here, we proceed again as above by selecting all the rows of this join columns table where nbNewsShare >= 500. They represent our news propagation with high number of shared.

In [499]:
Newsprop1 = '''SELECT newsLabels.newsId as 'News Id', newsLabels.label as 'News label',
                    newsPropagation.usersIdsFollowers,
                    newsPropagation.nbNewsShare  as 'Number of Time Shared',
                    newsPropagation.nbNewsView as 'Number of Time View'
                    FROM newsLabels
                    JOIN newsPropagation
                    ON newsLabels.newsId = newsPropagation.newsId
                    where  newsPropagation.nbNewsShare >= 500
                    order by  newsPropagation.nbNewsShare desc;
                    '''

runQuery(Newsprop1)

,News Id,News label,usersIdsFollowers,Number of Time Shared,Number of Time View
0,n178,2,"['u10', 'u11', 'u16', 'u31', 'u35', 'u45', 'u7...",3437,80730
1,n227,1,"['u7', 'u19', 'u22', 'u51', 'u62', 'u63', 'u79...",3383,85732
2,n233,1,"['u4', 'u30', 'u31', 'u34', 'u45', 'u47', 'u82...",1573,41917
3,n155,2,"['u6', 'u14', 'u24', 'u33', 'u41', 'u42', 'u89...",1438,34959
4,n37,0,"['u23', 'u38', 'u40', 'u74', 'u76', 'u77', 'u1...",1044,27028
5,n162,1,"['u3', 'u10', 'u52', 'u53', 'u72', 'u141', 'u1...",1030,21775
6,n128,2,"['u14', 'u31', 'u36', 'u37', 'u48', 'u113', 'u...",1029,24872
7,n135,1,"['u31', 'u33', 'u37', 'u45', 'u60', 'u92', 'u1...",824,19944
8,n145,2,"['u14', 'u17', 'u27', 'u145', 'u148', 'u173', ...",682,19345
9,n204,1,"['u46', 'u50', 'u122', 'u163', 'u174', 'u195',...",638,17420


Here, we proceed again as above by selecting all the rows of this join columns table where nbNewsShare <= 6. They represent our news propagation with low number of shared.

In [500]:
Newsprop2 = '''SELECT newsLabels.newsId as 'News Id', newsLabels.label as 'News label',
                    newsPropagation.usersIdsFollowers,
                    newsPropagation.nbNewsShare  as 'Number of Time Shared',
                    newsPropagation.nbNewsView as 'Number of Time View'
                    FROM newsLabels
                    JOIN newsPropagation
                    ON newsLabels.newsId = newsPropagation.newsId
                    where  newsPropagation.nbNewsShare < 10
                    order by  newsPropagation.nbNewsShare desc;
                    '''

runQuery(Newsprop2)

,News Id,News label,usersIdsFollowers,Number of Time Shared,Number of Time View
0,n228,2,"['u1399', 'u2972', 'u4505', 'u15418', 'u19883'...",9,171
1,n83,2,"['u354', 'u2340', 'u5571', 'u9882', 'u15723', ...",9,120
2,n184,1,"['u358', 'u868', 'u4763', 'u5450', 'u10049', '...",8,167
3,n38,0,"['u1995', 'u4944', 'u7020', 'u10850', 'u12996'...",8,96
4,n111,0,"['u301', 'u6890', 'u12883', 'u13020', 'u19752'...",7,74
5,n210,1,"['u4755', 'u10230', 'u15320', 'u18154', 'u2169...",7,274
6,n222,1,"['u26', 'u1881', 'u11063', 'u16960', 'u20378',...",7,237
7,n237,1,"['u4749', 'u7501', 'u12639', 'u12977', 'u13732...",7,173
8,n239,1,"['u1476', 'u12733', 'u16845', 'u16960', 'u1806...",7,50
9,n49,0,"['u4306', 'u6748', 'u18923', 'u20921', 'u22010...",7,75


In [ ]:
conn.commit()
con.close()      #Closing Connection